# Measurement Framework Libray (mflib)
Add measurement capabilities while creating new slice.

Currently the mflib branch must be installed for fabrictestbed-extensions. If you are using the a local mflib.py file you can use the fablib1.1.4dev1 branch. 
 

git clone https://github.com/fabric-testbed/fabrictestbed-extensions.git   
cd fabrictestbed-extensions  
git checkout mflib  
pip install --user .  



## Setup Environment

In [ ]:
import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']=<INSERT_YOUR_FABRIC_USERNAME>
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/fabric_bastion_key'

# Set the keypair FABRIC will install in your slice. 
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

# If your slice private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase. Press enter for no passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()
import json
import traceback

from fabrictestbed_extensions.fablib.fablib import fablib


slice_name = "MFMAX8"
site = 'MAX'
node1_name = 'Node1'
node2_name = 'Node2'
network_name='net1'
node1_nic_name = 'nic1'
node2_nic_name = 'nic2'
image = 'default_ubuntu_20'
image_type = 'qcow2'
cores = 2
ram = 8
disk = 10

## Import mflib and create new mflib object.

### Choose if you are using mflib branch code or using a local copy of mflib.py

In [ ]:
# If you installed the mflib branch you can use:
from fabrictestbed_extensions.mflib.mflib import mflib
# If you are using a local copy of the mflib.py file use:
#from mflib import mflib


In [ ]:
mf = mflib()

## Add measurement node to slice topology and submit the slice.

In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)

    # Node1
    node1 = slice.add_node(name=node1_name, site=site)
    node1.set_capacities(cores=cores, ram=ram, disk=disk)
    node1.set_image(image)
    iface1 = node1.add_component(model='NIC_Basic', name=node1_nic_name).get_interfaces()[0]
   
    # Node2
    node2 = slice.add_node(name=node2_name, site=site)
    node2.set_capacities(cores=cores, ram=ram, disk=disk)
    node2.set_image(image)
    iface2 = node2.add_component(model='NIC_Basic', name=node2_nic_name).get_interfaces()[0]
    
    # Network
    net1 = slice.add_l2network(name=network_name, interfaces=[iface1, iface2])
    
    # Add measurement node to topology.
    mf.addMeasNode(slice)
    # Submit Slice Request
    print("Submitting the new slice...")
    slice.submit()
    print("Slice creation done.")

except Exception as e:
    print(f"Slice Fail: {e}")
    traceback.print_exc()

## Initialize the measurement capabilities to the slice. 

In [ ]:
mf.init(slice_name)

## Instrumentize the slice to start the collection of metrics.

In [ ]:
mf.instrumentize()

## Get Prometheus Access Info
To access the Prometheus Metrics go to the grafana interface.   
From your local machine, tunnel into the measure node using `ssh -L 10010:localhost:443 -F <fabric-ssh-config-file> -i <your portal_slice_id_rsa-file> <slice-username>@<meas_node-ip>`   
Browse to [https://localhost:10010/grafana/](https://localhost:10010/grafana/)    
You will need to accept the self-signed certificate.   
There is a bug in the url, once you have accepted the certificate you will have to add the :10010 back to the url.  
The Grafana view opens to the default user. To login as and admin click on the door icon at the bottom left and enter u: admin  p: <grafana_admin_password (see below)>
    

In [ ]:
# The prometheus service was created by the mf.instrumentize call.
mf.info("prometheus")

In [ ]:
# To get specific info use:
data = {}
data["grafana_admin_password"] = True
info_results = mf.info("prometheus", data)
print(info_results["grafana_admin_password"])

In [ ]:
# Elk requires ht access user and pass
data = {}
data["ht_password"] = True
data["ht_user"] = True
info_results = mf.info("prometheus", data)
print(info_results)

## Explore User Services

In [ ]:
# Prometheus service was created during the intrumentize call.
mf.info("prometheus")

In [ ]:
# Create the overview service - used for debugging.
mf.create("overview")

In [ ]:
# Get the default overview information.
mf.info("overview")

In [ ]:
# Request the returned readmes in a certain format.
data = {}
data["readme_format"] = "combined"
mf.info("overview", data)

# Helpers for standard slice actions.
## Helper to get slice information and login commands.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        print(f"   Cores             : {node.get_cores()}")
        print(f"   RAM               : {node.get_ram()}")
        print(f"   Disk              : {node.get_disk()}")
        print(f"   Image             : {node.get_image()}")
        print(f"   Image Type        : {node.get_image_type()}")
        print(f"   Host              : {node.get_host()}")
        print(f"   Site              : {node.get_site()}")
        print(f"   Management IP     : {node.get_management_ip()}")
        print(f"   Reservation ID    : {node.get_reservation_id()}")
        print(f"   Reservation State : {node.get_reservation_state()}")
        print(f"   SSH Command       : {node.get_ssh_command()}")
        print(f"   Components        :  ")
        for component in node.get_components():
            print(f"      Name             : {component.get_name()}")
            print(f"      Details          : {component.get_details()}")
            print(f"      Disk (G)         : {component.get_disk()}")
            print(f"      Units            : {component.get_unit()}")
            print(f"      PCI Address      : {component.get_pci_addr()}")
            print(f"      Model            : {component.get_model()}")
            print(f"      Type             : {component.get_type()}") 
        print(f"   Interfaces        :  ")
        for interface in node.get_interfaces():
            print(f"       Name                : {interface.get_name()}")
            print(f"           Bandwidth           : {interface.get_bandwidth()}")
            print(f"           VLAN                : {interface.get_vlan()}")  
            print(f"           MAC                 : {interface.get_mac()}") 
            print(f"           OS iface name       : {interface.get_os_interface()}")
    for network in slice.get_l2networks():
        print("Network:")
        print(f"    Name:            {network.get_name()}")
    print(f"Interface Map: {slice.get_interface_map()}")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

## Check/extend slice life.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"Lease End         : {slice.get_lease_end()}")
       
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
import datetime

#Extend slice
end_date = (datetime.datetime.now() + datetime.timedelta(days=6)).strftime("%Y-%m-%d %H:%M:%S")

try:
    slice = fablib.get_slice(name=slice_name)
    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")

## Delete Slice

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()